In [1]:
import pandas as pd
import numpy as np


In [12]:
df1 = pd.read_csv("Tweet_Asterix.csv", sep = ";", usecols= ["created_at", "id", "text"])
print("Shape : ", df1.shape)
print("Colonnes : ", df1.columns)
df1.head()

Shape :  (1387, 3)
Colonnes :  Index(['created_at', 'id', 'text'], dtype='object')


,created_at,id,text
0,2023-02-08 13:46:53+00:00,1623317443903987713,Très bon retour spectateurs Allociné pour #ali...
1,2023-02-08 13:42:54+00:00,1623316442069233664,"J’vous jure c’est ni une aventure, ni une comé..."
2,2023-02-08 13:37:09+00:00,1623314995378356230,📚 Dans les librairies\nÀ découvrir dès aujourd...
3,2023-02-08 13:21:26+00:00,1623311040099262466,"@CarrefourFrance Bonjour, je participe pour te..."
4,2023-02-08 12:58:20+00:00,1623305227460919297,@CarrefourFrance #AsterixEtObelixLEmpireDuMili...


In [14]:
# We remove hours
def date_format(df):
    df["created_at"] = df["created_at"].str[:10]
    df = df.rename(columns = { "created_at" :"Date" })
    return df

df1 = date_format(df1)
df1.head()

,Date,id,text
0,2023-02-08,1623317443903987713,Très bon retour spectateurs Allociné pour #ali...
1,2023-02-08,1623316442069233664,"J’vous jure c’est ni une aventure, ni une comé..."
2,2023-02-08,1623314995378356230,📚 Dans les librairies\nÀ découvrir dès aujourd...
3,2023-02-08,1623311040099262466,"@CarrefourFrance Bonjour, je participe pour te..."
4,2023-02-08,1623305227460919297,@CarrefourFrance #AsterixEtObelixLEmpireDuMili...


In [7]:
#We check duplicata :

def check_duplicate(df):
    return df.duplicated().any()

check_duplicate(df1)

False